In [1]:
!nvidia-smi

Fri Jun 20 17:21:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.80                 Driver Version: 576.80         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   59C    P8              5W /   10W |     323MiB /   6141MiB |     85%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import cv2
import numpy as np
import os
import torch
from ultralytics import YOLO
import shutil
import tkinter as tk
import time
import math
import torch


In [3]:
#check whether cuda is enabled
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
# set device to cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

True
0
1
NVIDIA GeForce RTX 4050 Laptop GPU
cuda


In [4]:
classes = ['left_AP', 'right_AP', 'left_outlet', 'right_outlet']
# define segmentation method
def crop_segments(results, image):
    '''
    crop out the segment of the image

    Args:
        results: the results of the segmentation
        image: the image
    
    return:
        cropped_segments: list containing set of cropped segment information
    '''
    cropped_segments = []
    for result in results:
        if result.masks is not None:
            masks = result.masks.data.cpu().numpy()
            
            for i, mask in enumerate(masks):
                 # resize mask to match image dimensions
                if mask.shape != image.shape[:2]:
                    mask = cv2.resize(mask, (image.shape[1], image.shape[0]))

                # find bounding box of the mask
                coords = np.column_stack(np.where(mask > 0.5))
                if len(coords) > 0:
                    y1, x1 = coords.min(axis=0)
                    y2, x2 = coords.max(axis=0)

                    # crop the region
                    cropped_image = image[y1:y2+1, x1:x2+1]
                    cropped_mask = mask[y1:y2+1, x1:x2+1]

                    # Create image w black background
                    masked_image = cropped_image.copy()
                    # Set background pixels (where mask is False) to black
                    masked_image[cropped_mask <= 0.5] = [0, 0, 0]

                    cropped_segments.append({
                        'image': masked_image,
                        'original': cropped_image,
                        'mask': cropped_mask,
                        'index': i,
                        'bbox': (x1, y1, x2, y2)
                    })
    return cropped_segments

# save image method

def save_cropped_segments(segments, image_name, output_dir):
    '''save cropped segments to output_dir'''
    os.makedirs(output_dir, exist_ok=True)

    for segment in segments:
        filename = image_name.split('.')[0] + '_mask' + '.png'
        filepath = os.path.join(output_dir, filename)

        # Save image with black background
        cv2.imwrite(filepath, segment['image'])
        print(f'Saved: {filepath}')

def resize_with_aspect_ratio(image, width=None, height=None, inter=cv2.INTER_AREA):
    '''resize image with aspect ratio'''    
    # get the current width and height
    w, h = image.shape[1], image.shape[0]
    
    # calculate the scaling factor
    if width is None and height is None:
        return image
    if width is None:   
        scale = height / h
        dim = (int(w * scale), height)
    else:
        scale = width / w
        dim = (width, int(h * scale))

    return cv2.resize(image, dim, interpolation=inter)



In [ ]:
# combine test frames to form a new video
full_img_paths = []
os.makedirs('test_videos', exist_ok=True)

for class_name in classes:
    image_folder = os.path.join('test/test', class_name)
    curr_images = [img for img in os.listdir(image_folder) if img.endswith('.jpg')]
    full_img_paths.extend([os.path.join(image_folder, img) for img in curr_images])

image_folder = os.path.join('test/test', classes[0])
frame = cv2.imread(full_img_paths[0])
height, width, layers = frame.shape
video_writer = cv2.VideoWriter('test_videos/test_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 10, (width, height))

for img_path in full_img_paths:
    frame = cv2.imread(img_path)
    if frame is not None:
        # Ensure frame has correct dimensions
        if frame.shape[:2] == (height, width):
            video_writer.write(frame)
        else:
            # Resize if dimensions don't match
            frame_resized = cv2.resize(frame, (width, height))
            video_writer.write(frame_resized)
    else:
        print(f"Warning: Could not read image {img_path}")

video_writer.release()
print(f"Video created with {len(full_img_paths)} frames")

### Realtime Segmentation ###

In [8]:
# train 5 is model path to segmentation model
def real_time_segment(input_video, output_video, model_path=None, display_frame=False, conf_threshold=0.5):
    cap = cv2.VideoCapture(input_video)

    assert cap.isOpened(), 'Error reading video file'

    os.makedirs(os.path.dirname(output_video), exist_ok=True)

    # Video Writer
    w, h, fps = (int(cap.get(x))for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))


    # Initialze instance segmentation object
    show = False
    if display_frame:
        show = True

    isegment = solutions.InstanceSegmentation(
        show=show, # display output
        model=model_path, # path to model weights
        device=[0],
        conf=conf_threshold,
        verbose=False
    )

    # process video
    frame_count = 0
    processed_count = 0

    
    start_time = time.time()

    try:
        while cap.isOpened():
            success, im0 = cap.read()

            if not success:
                print('Video frame is empty or video processing successfully completed')
                break

            frame_count += 1

            # process frame
            processed_frame = isegment(im0)

            # write processed frame to output video
            if processed_frame is not None:
                video_writer.write(processed_frame.plot_im)
                processed_count += 1
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        end_time = time.time()
        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()
        print(f'Input frames: {frame_count}')
        print(f'Processed frames: {processed_count}')
        print(f'Output saved to: {output_video}')
        print(f'Inference speed: {((end_time - start_time) * 10**3) / processed_count} ms per frame')
        print('Video processing completed')

In [9]:
real_time_segment(input_video='input_videos/test_video2.mp4', output_video='output_videos/test_video2_seg.mp4', model_path='runs/segment/train5/weights/best.pt')

Ultralytics Solutions:  {'source': None, 'model': 'runs/segment/train5/weights/best.pt', 'classes': None, 'show_conf': True, 'show_labels': True, 'region': None, 'colormap': 21, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': False, 'iou': 0.7, 'conf': 0.5, 'device': [0], 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': False, 'data': 'images'}
WARNING no tracks found!
WARNING No masks detected! Ensure you're using a supported Ultralytics segmentation model.
WARNING no tracks found!
WARNING No masks detected! Ensure you're using a supported Ultralytics segmentation model.
WARNING no tracks found!
WARNING No masks detected! Ensure you're using a supported Ultralytics seg

### Real Time Detection ###


In [18]:
def real_time_detect(det_model_path, input_video, output_video, output_fps=None, display_frame=False, conf_threshold=0.25):
    det_model = YOLO(det_model_path)
    
    os.makedirs(os.path.dirname(output_video), exist_ok=True)
    
    cap = cv2.VideoCapture(input_video)
    assert cap.isOpened(), 'Error reading video file'

    # get video properties
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if output_fps is None:
        output_fps = original_fps
    

    print(f"Input: {width} x {height} at {original_fps} fps, {total_frames} frames")
    print(f"Output: {width} x {height} at {output_fps} fps")

    # intialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video, fourcc, output_fps, (width, height))

    if not video_writer.isOpened():
        print('Error opening video writer')
        cap.release()
        return
    
    frame_count = 0
    processed_count = 0
    start_time = time.time()
    
    # process video
    try:
        while cap.isOpened():
            success, frame = cap.read()

            if not success:
                print('Video frame is empty or video processing successfully completed')
                break
                
            frame_count += 1

            # resize frame if needed
            if frame.shape[0] != height or frame.shape[1] != width:
                frame = cv2.resize(frame, (width, height))

            # process frame
            results = det_model.predict(frame, verbose=False, conf=conf_threshold) 

            # draw results on frame
            annotated_frame = results[0].plot()

            # ensure annotated frame is the same size as the original frame
            if annotated_frame.shape[0] != height or annotated_frame.shape[1] != width:
                annotated_frame = cv2.resize(annotated_frame, (width, height))

            # write frame to output video
            video_writer.write(annotated_frame)
            processed_count += 1
            
            # print progress
            if frame_count % 30 == 0:
                print(f"Processed {processed_count} frames / {frame_count} total")
            
            # Display the frame
            if display_frame:
                cv2.imshow('YOLO11 Real-time', annotated_frame)
                key = cv2.waitKey(1) & 0xFF

                if key == ord('q'):
                    print('Processing stopped by user')
                    break
                elif key == ord('p'):
                    print('Processing paused, press any key to resume')
                    cv2.waitKey(0)
                    print('Processing resumed')
           
    except Exception as e:
        print(f"Error processing frame {frame_count}: {e}")

    finally:
        end_time = time.time()
        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()
        print('Video processing completed')
        print(f'Input frames: {frame_count}')
        print(f'Processed frames: {processed_count}')
        print(f'Output saved to: {output_video}')
        print('Inference speed: ', ((end_time - start_time)* 10**3) / frame_count, 'ms/frame')

In [17]:
real_time_detect(det_model_path='runs/detect/train2/weights/best.pt', input_video='input_videos/test_video2.mp4', output_video='output_videos/test_video2_det.mp4', conf_threshold=0.732)

Input: 1080 x 1920 at 30.21165555233401 fps, 521 frames
Output: 1080 x 1920 at 30.21165555233401 fps
Processed 30 frames / 30 total
Processed 60 frames / 60 total
Processed 90 frames / 90 total
Processed 120 frames / 120 total
Processed 150 frames / 150 total
Processed 180 frames / 180 total
Processed 210 frames / 210 total
Processed 240 frames / 240 total
Processed 270 frames / 270 total
Processed 300 frames / 300 total
Processed 330 frames / 330 total
Processed 360 frames / 360 total
Processed 390 frames / 390 total
Processed 420 frames / 420 total
Processed 450 frames / 450 total
Processed 480 frames / 480 total
Processed 510 frames / 510 total
Video frame is empty or video processing successfully completed
Video processing completed
Input frames: 521
Processed frames: 521
Output saved to: output_videos/test_video2_det.mp4
Inference speed:  38.420639660445374 ms/frame


In [16]:
real_time_detect(det_model_path='runs/detect/train2/weights/best.pt', input_video='input_videos/test_video3.mp4', output_video='output_videos/test_video3_det.mp4', conf_threshold=0.732)

Input: 1080 x 1920 at 30.011688030155746 fps, 1035 frames
Output: 1080 x 1920 at 30.011688030155746 fps
Processed 30 frames / 30 total
Processed 60 frames / 60 total
Processed 90 frames / 90 total
Processed 120 frames / 120 total
Processed 150 frames / 150 total
Processed 180 frames / 180 total
Processed 210 frames / 210 total
Processed 240 frames / 240 total
Processed 270 frames / 270 total
Processed 300 frames / 300 total
Processed 330 frames / 330 total
Processed 360 frames / 360 total
Processed 390 frames / 390 total
Processed 420 frames / 420 total
Processed 450 frames / 450 total
Processed 480 frames / 480 total
Processed 510 frames / 510 total
Processed 540 frames / 540 total
Processed 570 frames / 570 total
Processed 600 frames / 600 total
Processed 630 frames / 630 total
Processed 660 frames / 660 total
Processed 690 frames / 690 total
Processed 720 frames / 720 total
Processed 750 frames / 750 total
Processed 780 frames / 780 total
Processed 810 frames / 810 total
Processed 8

In [19]:
real_time_detect(det_model_path='runs/detect/train_light/weights/best.pt', input_video='input_videos/test_video3.mp4', output_video='output_videos/test_video3_det.mp4')

Input: 1080 x 1920 at 30.011688030155746 fps, 1035 frames
Output: 1080 x 1920 at 30.011688030155746 fps
Processed 30 frames / 30 total
Processed 60 frames / 60 total
Processed 90 frames / 90 total
Processed 120 frames / 120 total
Processed 150 frames / 150 total
Processed 180 frames / 180 total
Processed 210 frames / 210 total
Processed 240 frames / 240 total
Processed 270 frames / 270 total
Processed 300 frames / 300 total
Processed 330 frames / 330 total
Processed 360 frames / 360 total
Processed 390 frames / 390 total
Processed 420 frames / 420 total
Processed 450 frames / 450 total
Processed 480 frames / 480 total
Processed 510 frames / 510 total
Processed 540 frames / 540 total
Processed 570 frames / 570 total
Processed 600 frames / 600 total
Processed 630 frames / 630 total
Processed 660 frames / 660 total
Processed 690 frames / 690 total
Processed 720 frames / 720 total
Processed 750 frames / 750 total
Processed 780 frames / 780 total
Processed 810 frames / 810 total
Processed 8

### Realtime Classification ###

In [ ]:
# crop bounding box
def crop_bounding_box(results, image):
    '''
        crops image based on bounding box returned by YOLOv11 model
        
        Args:
            results: YOLOv11 model output
            image: image to crop
            
        Returns:
            cropped: cropped image
    '''
    person_class_id = 0
    for result in results:
        if result.boxes is not None:
            classes = result.boxes.cls.cpu().numpy()
            boxes = result.boxes.xyxy.cpu().numpy()

            for i, class_id in enumerate(classes):
                if int(class_id) == person_class_id:
                    x1, y1, x2, y2 = map(int, boxes[i])
                    cropped = image[y1:y2, x1:x2]
                    return cropped
                
    print('No person detected in image')
    return image

# create augmentation pipeline
def crop_image(img, crop_percent):
    return img[int(crop_percent*img.shape[0]):int((1-crop_percent)*img.shape[0]), int(crop_percent*img.shape[1]):int((1-crop_percent)*img.shape[1])]

def draw_line(img):
    '''
        draws line down middle of image
        
        Args:
            img(cv2 image): image to draw line on
        
        Returns:
            None
    '''
    # get image dimensions
    h, w = img.shape[:2]

    # calc middle coordinate
    middle_x = w // 2

    # draw line down middle
    cv2.line(img, (middle_x, 0), (middle_x, h), (0, 255, 0), 2)

def real_time_classify(cls_model_path, input_video, output_video, det_model_path=None, output_fps=None, bounding_box_crop=False, 
                       fixed_crop=False, crop_percent=0.1, line=False, display_frame=False):
    '''
        Performs real-time classification on video using YOLOv11 model
        
        Args:
            cls_model_path: path to YOLOv11 classification model
            det_model_path: path to YOLOv11 detection model
            input_video: path to input video
            output_video: path to output video
            output_fps: output video fps, if None, use original fps
            bounding_box_crop: if True, crops bounding box around person, else crops around edges
            fixed_crop: if True, crops image to fixed size 
            crop_percent: percentage of image to crop (default 0.1)
            
        Returns: 
            None
    '''
    # load models
    cls_model = YOLO(cls_model_path)

    if det_model_path is not None:
        det_model = YOLO(det_model_path)
    
    os.makedirs(os.path.dirname(output_video), exist_ok=True)
    
    cap = cv2.VideoCapture(input_video)
    assert cap.isOpened(), 'Error reading video file'

    # get video properties
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if output_fps is None:
        output_fps = original_fps
    

    print(f"Input: {width} x {height} at {original_fps} fps, {total_frames} frames")
    print(f"Output: {width} x {height} at {output_fps} fps")

    # intialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video, fourcc, output_fps, (width, height))

    if not video_writer.isOpened():
        print('Error opening video writer')
        cap.release()
        return
    
    frame_count = 0
    processed_count = 0
    start_time = time.time()
    
    # process video
    try:
        while cap.isOpened():
            success, frame = cap.read()

            if not success:
                print('Video frame is empty or video processing successfully completed')
                break
                
            frame_count += 1

            if det_model_path is not None:
                # get bounding box results
                results = det_model(frame, verbose=False)
                frame = crop_bounding_box(results, frame)
            elif fixed_crop:
                crop_image(frame, crop_percent)

            if line:
                draw_line(frame)

            # resize frame if needed
            if frame.shape[0] != height or frame.shape[1] != width:
                frame = cv2.resize(frame, (width, height))

            # process frame
            results = cls_model(frame, verbose=False) 

            # draw results on frame
            annotated_frame = results[0].plot()

            # ensure annotated frame is the same size as the original frame
            if annotated_frame.shape[0] != height or annotated_frame.shape[1] != width:
                annotated_frame = cv2.resize(annotated_frame, (width, height))

            # write frame to output video
            video_writer.write(annotated_frame)
            processed_count += 1
            
            # print progress
            if frame_count % 30 == 0:
                print(f"Processed {processed_count} frames / {frame_count} total")
            
            if display_frame:
                final_frame = resize_with_aspect_ratio(annotated_frame, width=500, height=500)
                # Display the frame
                cv2.imshow('YOLO11 Real-time', final_frame)
                key = cv2.waitKey(1) & 0xFF

                if key == ord('q'):
                    print('Processing stopped by user')
                    break
                elif key == ord('p'):
                    print('Processing paused, press any key to resume')
                    cv2.waitKey(0)
                    print('Processing resumed')
           
    except Exception as e:
        print(f"Error processing frame {frame_count}: {e}")

    finally:
        end_time = time.time()
        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()
        print('Video processing completed')
        print(f'Input frames: {frame_count}')
        print(f'Processed frames: {processed_count}')
        print(f'Output saved to: {output_video}')
        print('Inference speed: ', ((end_time - start_time) * 10**3) / frame_count, 'ms/frame')


In [15]:

real_time_classify(cls_model_path='runs/classify/train96/weights/best.pt', det_model_path='yolo11n.pt', input_video='input_videos/test_video2.mp4', output_video='output_videos/test_video2_cls_bbox.mp4')

Input: 1080 x 1920 at 30.21165555233401 fps, 521 frames
Output: 1080 x 1920 at 30.21165555233401 fps
Error processing frame 1: name 'resize_with_aspect_ratio' is not defined
Video processing completed
Input frames: 1
Processed frames: 1
Output saved to: output_videos/test_video2_cls_bbox.mp4
Inference speed:  672.3356246948242 ms/frame


In [18]:
real_time_classify(cls_model_path='runs/classify/train80/weights/best.pt', input_video='input_videos/test_video2.mp4', output_video='output_videos/test_video2_cls.mp4')

Input: 1080 x 1920 at 30.21165555233401 fps, 521 frames
Output: 1080 x 1920 at 30.21165555233401 fps
Processed 30 frames / 30 total
Processed 60 frames / 60 total
Processed 90 frames / 90 total
Processed 120 frames / 120 total
Processed 150 frames / 150 total
Processed 180 frames / 180 total
Processed 210 frames / 210 total
Processed 240 frames / 240 total
Processed 270 frames / 270 total
Processed 300 frames / 300 total
Processed 330 frames / 330 total
Processed 360 frames / 360 total
Processed 390 frames / 390 total
Processed 420 frames / 420 total
Processed 450 frames / 450 total
Processed 480 frames / 480 total
Processed 510 frames / 510 total
Video frame is empty or video processing successfully completed
Video processing completed
Input frames: 521
Processed frames: 521
Output saved to: output_videos/test_video2_cls.mp4
Inference speed:  56.391574142075314 ms/frame


In [24]:
real_time_classify(cls_model_path='runs/classify/train80/weights/best.pt', input_video='input_videos/test_video3.mp4', output_video='output_videos/test_video3_cls_fixedcrop.mp4', fixed_crop=True, line=True)

Input: 1080 x 1920 at 30.011688030155746 fps, 1035 frames
Output: 1080 x 1920 at 30.011688030155746 fps
Processed 30 frames / 30 total
Processed 60 frames / 60 total
Processed 90 frames / 90 total
Processed 120 frames / 120 total
Processed 150 frames / 150 total
Processed 180 frames / 180 total
Processed 210 frames / 210 total
Processed 240 frames / 240 total
Processed 270 frames / 270 total
Processed 300 frames / 300 total
Processed 330 frames / 330 total
Processed 360 frames / 360 total
Processed 390 frames / 390 total
Processed 420 frames / 420 total
Processed 450 frames / 450 total
Processed 480 frames / 480 total
Processed 510 frames / 510 total
Processed 540 frames / 540 total
Processed 570 frames / 570 total
Processed 600 frames / 600 total
Processed 630 frames / 630 total
Processed 660 frames / 660 total
Processed 690 frames / 690 total
Processed 720 frames / 720 total
Processed 750 frames / 750 total
Processed 780 frames / 780 total
Processed 810 frames / 810 total
Processed 8

In [ ]:
real_time_classify(cls_model='runs/classify/runs/train35', input_video='input_videos/test_video3.mp4', output_video='output_videos/test_video3_cls_bbcrop.mp4')

Input: 1080 x 1920 at 30.011688030155746 fps, 1035 frames
Output: 1080 x 1920 at 30.011688030155746 fps
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
Processed 30 frames / 30 total
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping bounding box...
cropping boundi

### Pose Estimation Classifier ###

In [5]:
def calc_euclidean_distance(x1, y1, x2, y2):
    '''
        returns the euclidean distance between two points
        
        Args:
            x1 (float): x coordinate of first point
            y1 (float): y coordinate of first point
            x2 (float): x coordinate of second point
            y2 (float): y coordinate of second point
            
        Returns:
            float: euclidean distance between the two points
    '''
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def get_video_properties(cap):
    '''
        returns the video properties
        
        Args:
            cap (cv2.VideoCapture): video capture object
            
        Returns:
            tuple: fps, width, height, total_frames
    '''
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    return original_fps, width, height, total_frames

def add_text_to_frame(display_text, annotated_frame, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=1.5, thickness=2, bgr=(218, 232, 0)):
    '''
        adds text to a frame
        
        Args:
            display_text (str): text to be displayed
            font (int): font type
            font_scale (float): font scale
            thickness (int): thickness of the text
            annotated_frame (numpy.ndarray): frame to add text to
            bgr (tuple): background color of the text
    '''
    (text_width, text_height), baseline = cv2.getTextSize(display_text, font, font_scale, thickness)
    line_height = text_height + baseline + 5
    y_offset = 50

    for i, line in enumerate(display_text.split('\n')):
        cv2.putText(annotated_frame, line, 
                        org=(10, y_offset + i * line_height), 
                        fontFace=font, 
                        fontScale=font_scale, 
                        color=bgr, 
                        thickness=thickness)
        
def calc_intersection(box1, box2):
    '''
        calculates the intersection of two bounding boxes
        
        Args:
            box1 (tuple): bounding box coordinates (x1, y1, x2, y2)
            box2 (tuple): bounding box coordinates (x1, y1, x2, y2)
            
        Returns:
            float: intersection of the two bounding boxes
    '''

    x1, y1, x2, y2 = box1 
    x3, y3, x4, y4 = box2

    # calculate intersection area
    intersection_x = min(x2, x4) - max(x1, x3)
    intersection_y = min(y2, y4) - max(y1, y3)
    intersection_area = intersection_x * intersection_y

    return intersection_area
    
def get_classification(pose_model, det_model, frame):

    det_results = det_model(frame, verbose=False)
    pose_results = pose_model(frame, verbose=False)

    light_coordinates = [] # stores tuples of ((x_mid, y_mid, width, height))
    light_detected = False
    l_mid_x, l_mid_y = 0, 0 # midpoint of light bounding box
    annotated_frame = frame
    total_mid_x, total_mid_y = 0, 0

    if det_results and len(det_results) > 0:
        det_result = det_results[0]
        if det_result.boxes is not None and len(det_result.boxes) > 0:
            annotated_frame = det_result.plot() # plot detection results on frame
        
            # calculate average midpoint of all light detections
            class_indices = det_result.boxes.cls.int().cpu().numpy() if det_result.boxes.cls is not None else []
            for i, class_idx in enumerate(class_indices):
                class_name = det_model.names[class_idx]
                    
                if class_name == 'light':
                    light_coordinates.append(det_result.boxes.xywh[i].cpu().numpy())
                    light_detected = True
                    
    if light_detected:
                
        for l_x_mid, l_y_mid, _, _ in light_coordinates:
            total_mid_x += l_x_mid
            total_mid_y += l_y_mid
        l_mid_x, l_mid_y = total_mid_x / len(light_coordinates), total_mid_y / len(light_coordinates)
                
        if pose_results is not None and len(pose_results) > 0:

            total_left_shoulder_x, total_left_shoulder_y, total_right_shoulder_x, total_right_shoulder_y = 0, 0, 0, 0
            left_eye_present, right_eye_present = False, False # used to determine if AP or outlet
            valid_pose_detected = False
            pose_count = 0
            
            result = pose_results[0]
            annotated_frame = result.plot(img=annotated_frame) # plot pose estimation results on frame
            if result.keypoints is not None and result.keypoints.data is not None:
                keypoints_data = result.keypoints.data

                keypoints = keypoints_data[0] # take first person

                if keypoints.shape[0] >= 17: # ensure all keypoints present                   
                    left_shoulder_x, left_shoulder_y = float(keypoints[5][0]), float(keypoints[5][1])
                    right_shoulder_x, right_shoulder_y = float(keypoints[6][0]), float(keypoints[6][1])
                                
                    total_left_shoulder_x += left_shoulder_x
                    total_left_shoulder_y += left_shoulder_y
                    total_right_shoulder_x += right_shoulder_x 
                    total_right_shoulder_y += right_shoulder_y
                                
                    left_eye_conf = float(keypoints[1][2]) if keypoints.shape[0] > 1 else 0 
                    right_eye_conf = float(keypoints[2][2]) if keypoints.shape[0] > 2 else 0
                                    
                    left_eye_present = left_eye_conf > 0.3
                    right_eye_present = right_eye_conf > 0.3

                    pose_count += 1
                    valid_pose_detected = True
                                    
            if valid_pose_detected:
                # calculate average shoulder coordinates
                avg_left_shoulder_x = total_left_shoulder_x / pose_count
                avg_left_shoulder_y = total_left_shoulder_y / pose_count
                avg_right_shoulder_x = total_right_shoulder_x / pose_count                    
                avg_right_shoulder_y = total_right_shoulder_y / pose_count
                        
                # add to distance between shoulders
                dist_left = calc_euclidean_distance(avg_left_shoulder_x, avg_left_shoulder_y, l_mid_x, l_mid_y)
                dist_right = calc_euclidean_distance(avg_right_shoulder_x, avg_right_shoulder_y, l_mid_x, l_mid_y)

                # get final class
                if dist_left < dist_right:
                    class_name = 'left'
                else:
                    class_name = 'right'
                                
                if not left_eye_present or not right_eye_present:
                    class_name += '_outlet'
                else:
                    class_name += '_AP'

                display_text = (  
                    f'Class: {class_name}\n'
                    f'Left Shoulder Distance: {dist_left:.2f} p\n'
                    f'Right Shoulder Distance: {dist_right:.2f} p\n'
                    f'Left Eye Present: {left_eye_present}\n'
                    f'Right Eye Present: {right_eye_present}'
                )
                        
            else:
                display_text = 'Pose detected but no shoulders'
                class_name = 'unknown'
        else:    
            display_text = 'No valid pose detected'
            class_name = 'unknown'
    else:
        display_text = 'No light detected'
        class_name = 'unknown'

    return annotated_frame, display_text, class_name

def get_classification_v2(pose_model, det_model, frame):

    det_results = det_model(frame, verbose=False)
    pose_results = pose_model(frame, verbose=False)

    light_coordinates = [] # stores tuples of ((l_x1, l_y1, lx2, ly2), (x_mid, y_mid, width, height))
    light_detected = False
    l_mid_x, l_mid_y = 0, 0 # midpoint of light bounding box
    annotated_frame = frame
    total_mid_x, total_mid_y = 0, 0

    if det_results and len(det_results) > 0:
        det_result = det_results[0]
        if det_result.boxes is not None and len(det_result.boxes) > 0:
            annotated_frame = det_result.plot() # plot detection results on frame
        
            # calculate average midpoint of all light detections
            class_indices = det_result.boxes.cls.int().cpu().numpy() if det_result.boxes.cls is not None else []
            for i, class_idx in enumerate(class_indices):
                class_name = det_model.names[class_idx]
                    
                if class_name == 'light':
                    light_coordinates.append((det_result.boxes.xyxy[i].cpu().numpy(), det_result.boxes.xywh[i].cpu().numpy()))
                    light_detected = True
                    
    if light_detected:

        if pose_results is not None and len(pose_results) > 0:

            # get person bounding box
            result = pose_results[0]
            annotated_frame = result.plot(img=annotated_frame) # plot pose estimation results on frame

            if result.boxes is not None and len(result.boxes) > 0:
                p_x1, p_y1, p_x2, p_y2 = result.boxes.xyxy[0]

            # calculate average midpoint of all light detections
            intersect_count = 0
            for (l_x1, l_y1, l_x2, l_y2), (l_x_mid, l_y_mid, w, h) in light_coordinates:
                intersection_area = calc_intersection((l_x1, l_y1, l_x2, l_y2), (p_x1, p_y1, p_x2, p_y2)) # check for intersection
                if intersection_area > 0: # if intersection area is greater than 0, add to total
                    total_mid_x += l_x_mid
                    total_mid_y += l_y_mid
                    intersect_count += 1

            if intersect_count == 0: # no x-ray light means no class
                class_name = 'unknown'
                display_text = 'No x-ray light detected'
                return annotated_frame, display_text, class_name
            
            l_mid_x, l_mid_y = total_mid_x / intersect_count, total_mid_y / intersect_count # calc

            total_left_shoulder_x, total_left_shoulder_y, total_right_shoulder_x, total_right_shoulder_y = 0, 0, 0, 0
            left_eye_present, right_eye_present = False, False # used to determine if AP or outlet
            valid_pose_detected = False
            pose_count = 0
            
            if result.keypoints is not None and result.keypoints.data is not None:
                keypoints_data = result.keypoints.data

                keypoints = keypoints_data[0] # take first person

                if keypoints.shape[0] >= 17: # ensure all keypoints present                   
                    left_shoulder_x, left_shoulder_y = float(keypoints[5][0]), float(keypoints[5][1])
                    right_shoulder_x, right_shoulder_y = float(keypoints[6][0]), float(keypoints[6][1])
                                
                    total_left_shoulder_x += left_shoulder_x
                    total_left_shoulder_y += left_shoulder_y
                    total_right_shoulder_x += right_shoulder_x 
                    total_right_shoulder_y += right_shoulder_y
                                
                    left_eye_conf = float(keypoints[1][2]) if keypoints.shape[0] > 1 else 0 
                    right_eye_conf = float(keypoints[2][2]) if keypoints.shape[0] > 2 else 0
                                    
                    left_eye_present = left_eye_conf > 0.3
                    right_eye_present = right_eye_conf > 0.3

                    pose_count += 1
                    valid_pose_detected = True
                                    
            if valid_pose_detected:
                # calculate average shoulder coordinates
                avg_left_shoulder_x = total_left_shoulder_x / pose_count
                avg_left_shoulder_y = total_left_shoulder_y / pose_count
                avg_right_shoulder_x = total_right_shoulder_x / pose_count                    
                avg_right_shoulder_y = total_right_shoulder_y / pose_count
                        
                # add to distance between shoulders
                dist_left = calc_euclidean_distance(avg_left_shoulder_x, avg_left_shoulder_y, l_mid_x, l_mid_y)
                dist_right = calc_euclidean_distance(avg_right_shoulder_x, avg_right_shoulder_y, l_mid_x, l_mid_y)

                # get final class
                if dist_left < dist_right:
                    class_name = 'left'
                else:
                    class_name = 'right'
                                
                if not left_eye_present or not right_eye_present:
                    class_name += '_outlet'
                else:
                    class_name += '_AP'

                display_text = (  
                    f'Class: {class_name}\n'
                    f'Left Shoulder Distance: {dist_left:.2f} p\n'
                    f'Right Shoulder Distance: {dist_right:.2f} p\n'
                    f'Left Eye Present: {left_eye_present}\n'
                    f'Right Eye Present: {right_eye_present}'
                )
                        
            else:
                display_text = 'Pose detected but no shoulders'
                class_name = 'unknown'
        else:    
            display_text = 'No valid pose detected'
            class_name = 'unknown'
    else:
        display_text = 'No light detected'
        class_name = 'unknown'

    return annotated_frame, display_text, class_name




def video_pose_light_classify(pose_model_path, det_model_path, video_path, output_path, display_frame=False):
    '''
        real time classification using pose estimation and light detection
        
        Args:
            pose_model_path (str): path to the pose estimation model
            det_model_path (str): path to the light detection model
            video_path (str): path to the video file
            output_path (str): path to the output video file
            display_frame (bool): whether to display the frame or not

        Returns:
            None
    '''
    # load models
    pose_model = YOLO(pose_model_path)
    det_model = YOLO(det_model_path)

    # open video capture
    cap = cv2.VideoCapture(video_path)

    # get video properties
    original_fps, width, height, total_frames = get_video_properties(cap)

    print(f"Video properties: fps={original_fps}, width={width}, height={height}, total frames={total_frames}")

    # initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_path, fourcc, original_fps, (width, height))

    if not video_writer.isOpened():
        print('Error opening video writer')
        cap.release()
        return

    frame_count = 0
    processed_count = 0
    start_time = time.time()

    # process video
    try:
        while cap.isOpened():
            success, frame = cap.read()

            if not success:
                print('Video frame is empty or video processing successfully completed')
                break

            frame_count += 1

            annotated_frame, display_text, class_name = get_classification_v2(pose_model, det_model, frame)
                
            # add text to frame
            add_text_to_frame(display_text=display_text, annotated_frame=annotated_frame)
                
            if annotated_frame.shape[:2] != (height, width):
                annotated_frame = cv2.resize(annotated_frame, (width, height))
        
            video_writer.write(annotated_frame) # save frame to output video
            processed_count += 1

            # print progress
            if frame_count % 30 == 0:
                print(f"Processed {processed_count} frames / {frame_count} total")

            # display frame in real time if requested
            if display_frame: 
                final_frame = resize_with_aspect_ratio(annotated_frame, width=500, height=500)
                
                cv2.imshow('YOLO11 Real-time', final_frame)
                key = cv2.waitKey(1) & 0xFF

                if key == ord('q'):
                    print('Processing stopped by user')
                    break
                elif key == ord('p'):
                    print('Processing paused, press any key to resume')
                    cv2.waitKey(0)
                    print('Processing resumed')

    except Exception as e:
        e.with_traceback(e)
    finally:
        end_time = time.time()
        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()
        print('Video processing completed')
        print(f'Input frames: {frame_count}')
        print(f'Processed frames: {processed_count}')
        print(f'Output saved to: {output_path}')
        print('Inference speed: ', ((end_time - start_time) * 10**3) / frame_count, 'ms/frame')

            


In [13]:
def image_pose_light_classify(pose_model_path, det_model_path, img_path, output_path):
    '''
        real time classification using pose estimation and x-ray light detection. saves images in input_path, and saves to output_path
        
        Args:
            pose_model_path (str): path to the pose estimation model
            det_model_path (str): path to the x-ray light detection model
            images_path (str): path to the images directory
            output_path (str): path to the output video file
            display_frame (bool): whether to display the frame or not

        Returns:    
            None
    '''

    # load models
    
    pose_model =  YOLO(pose_model_path)
    det_model = YOLO(det_model_path)

    # iterate through images and apply pose_estimation and light detection to classify images

    # load image
    frame = cv2.imread(img_path)

    annotated_frame, display_text, class_name = get_classification(pose_model, det_model, frame)

    add_text_to_frame(font_scale=0.5, display_text=display_text, annotated_frame=annotated_frame)

    # save image
    cv2.imwrite(output_path, annotated_frame)

    return class_name

def pose_detect_batch_classify(input_dir, output_dir, pose_model_path, det_model_path, classes):
    '''
        classify images in input_dir using pose estimation and x-ray light detection. saves images in output_dir 
        and prints the per class accuracy
        
        Args:
            input_dir (str): path to the input directory
            output_dir (str): path to the output directory
            pose_model_path (str): path to the pose estimation model
            det_model_path (str): path to the x-ray light detection model

        Returns:    
            None
    '''
    total_processed = 0
    total_correct = 0
    for class_name in classes:
        curr_input_path = os.path.join(input_dir, class_name)
        curr_output_path = os.path.join(output_dir, class_name)
        shutil.rmtree(curr_output_path, ignore_errors=True)
        os.makedirs(curr_output_path, exist_ok=True)

        print('Processing class:', class_name, 'from', input_dir)
    
        imgs = [img for img in os.listdir(curr_input_path) if img.endswith('.jpg')]

        total_images = len(imgs)
        correct_classified_count = 0
        processed_count = 0 

        print(f'Total images: {total_images}')
        for img in imgs:
            img_path = os.path.join(curr_input_path, img)
            img_output_path = os.path.join(curr_output_path, img)
            predicted_class_name = image_pose_light_classify(pose_model_path=pose_model_path, det_model_path=det_model_path, 
                                                img_path=img_path, output_path=img_output_path)
            
            if predicted_class_name == class_name:
                correct_classified_count += 1

            processed_count += 1

            if processed_count % 30 == 0:
                print(f'Processed {processed_count} of {total_images}')
        
        print(f'Correctly classified images: {correct_classified_count}')
        print(f'Class accuracy: {correct_classified_count / total_images}')
        
        total_processed += total_images
        total_correct += correct_classified_count
    
    print(f'Total processed: {total_processed}')
    print(f'Total correct: {total_correct}')
    print(f'Total Accuracy: {total_correct / total_processed}')

In [7]:
video_pose_light_classify(det_model_path='runs/detect/train_light35deg/weights/best.pt', pose_model_path='yolo11l-pose.pt', video_path='input_videos/test_video2.mp4', output_path='output_videos/test_video2_light.mp4', display_frame=True)

Video properties: fps=30.21165555233401, width=1080, height=1920, total frames=521
Processed 30 frames / 30 total
Processed 60 frames / 60 total
Processed 90 frames / 90 total
Processed 120 frames / 120 total
Processed 150 frames / 150 total
Processed 180 frames / 180 total
Processed 210 frames / 210 total
Processed 240 frames / 240 total
Processed 270 frames / 270 total
Processed 300 frames / 300 total
Processed 330 frames / 330 total
Processed 360 frames / 360 total
Processed 390 frames / 390 total
Processed 420 frames / 420 total
Processed 450 frames / 450 total
Processed 480 frames / 480 total
Processed 510 frames / 510 total
Video frame is empty or video processing successfully completed
Video processing completed
Input frames: 521
Processed frames: 521
Output saved to: output_videos/test_video2_light.mp4
Inference speed:  131.9465261960899 ms/frame


In [8]:
video_pose_light_classify(det_model_path='runs/detect/train_light/weights/best.pt', pose_model_path='yolo11l-pose.pt', video_path='input_videos/test_video3.mp4', output_path='output_videos/test_video3_light.mp4', display_frame=True)

Video properties: fps=30.011688030155746, width=1080, height=1920, total frames=1035
Processed 30 frames / 30 total
Processed 60 frames / 60 total
Processed 90 frames / 90 total
Processed 120 frames / 120 total
Processed 150 frames / 150 total
Processed 180 frames / 180 total
Processed 210 frames / 210 total
Processed 240 frames / 240 total
Processed 270 frames / 270 total
Processed 300 frames / 300 total
Processed 330 frames / 330 total
Processed 360 frames / 360 total
Processed 390 frames / 390 total
Processed 420 frames / 420 total
Processed 450 frames / 450 total
Processed 480 frames / 480 total
Processed 510 frames / 510 total
Processed 540 frames / 540 total
Processed 570 frames / 570 total
Processed 600 frames / 600 total
Processed 630 frames / 630 total
Processed 660 frames / 660 total
Processed 690 frames / 690 total
Processed 720 frames / 720 total
Processed 750 frames / 750 total
Processed 780 frames / 780 total
Processed 810 frames / 810 total
Processed 840 frames / 840 tot

In [9]:
video_pose_light_classify(det_model_path='runs/detect/train_light/weights/best.pt', pose_model_path='yolo11l-pose.pt', video_path='input_videos/test_video4.mp4', output_path='output_videos/test_video4_light.mp4', display_frame=True)

Video properties: fps=30.0, width=1080, height=1920, total frames=333
Processed 30 frames / 30 total
Processed 60 frames / 60 total
Processed 90 frames / 90 total
Processed 120 frames / 120 total
Processed 150 frames / 150 total
Processed 180 frames / 180 total
Processed 210 frames / 210 total
Processed 240 frames / 240 total
Processed 270 frames / 270 total
Processed 300 frames / 300 total
Processed 330 frames / 330 total
Video frame is empty or video processing successfully completed
Video processing completed
Input frames: 333
Processed frames: 333
Output saved to: output_videos/test_video4_light.mp4
Inference speed:  91.70113741098581 ms/frame


In [10]:
video_pose_light_classify(det_model_path='runs/detect/train_light/weights/best.pt', pose_model_path='yolo11l-pose.pt', video_path='input_videos/test_video5.mp4', output_path='output_videos/test_video5_light.mp4', display_frame=True)

Video properties: fps=30.0, width=1080, height=1920, total frames=337
Processed 30 frames / 30 total
Processed 60 frames / 60 total
Processed 90 frames / 90 total
Processed 120 frames / 120 total
Processed 150 frames / 150 total
Processed 180 frames / 180 total
Processed 210 frames / 210 total
Processed 240 frames / 240 total
Processed 270 frames / 270 total
Processed 300 frames / 300 total
Processed 330 frames / 330 total
Video frame is empty or video processing successfully completed
Video processing completed
Input frames: 337
Processed frames: 337
Output saved to: output_videos/test_video5_light.mp4
Inference speed:  103.86039029243085 ms/frame


In [17]:
# try on all training images
classes = ['left_outlet', 'right_outlet']

pose_detect_batch_classify(classes=classes, input_dir='original_data/train', output_dir='pose_detect_images/train_posex', pose_model_path='yolo11x-pose.pt', 
                           det_model_path='runs/detect/train_light35deg/weights/best.pt')

Processing class: left_outlet from original_data/train
Total images: 402
Processed 30 of 402
Processed 60 of 402
Processed 90 of 402
Processed 120 of 402
Processed 150 of 402
Processed 180 of 402
Processed 210 of 402
Processed 240 of 402
Processed 270 of 402
Processed 300 of 402
Processed 330 of 402
Processed 360 of 402
Processed 390 of 402
Correctly classified images: 336
Class accuracy: 0.835820895522388
Processing class: right_outlet from original_data/train
Total images: 419
Processed 30 of 419
Processed 60 of 419
Processed 90 of 419
Processed 120 of 419
Processed 150 of 419
Processed 180 of 419
Processed 210 of 419
Processed 240 of 419
Processed 270 of 419
Processed 300 of 419
Processed 330 of 419
Processed 360 of 419
Processed 390 of 419
Correctly classified images: 417
Class accuracy: 0.9952267303102625
Total processed: 821
Total correct: 753
Total Accuracy: 0.9171741778319124


In [ ]:
# try on all test images
classes = ['left_AP', 'right_AP', 'left_outlet', 'right_outlet']

for class_name in classes:
    input_dir = f'test/test/{class_name}'
    output_dir = f'pose_detect_images/test_posex/{class_name}'
    shutil.rmtree(output_dir, ignore_errors=True)
    os.makedirs(output_dir, exist_ok=True)

    print('Processing class:', class_name, 'from', input_dir)
   
    imgs = [img for img in os.listdir(input_dir) if img.endswith('.jpg')]

    total_images = len(imgs)
    correct_classified_count = 0
    processed_count = 0 

    print(f'Total images: {total_images}')
    for img in imgs:
        img_path = os.path.join(input_dir, img)
        output_path = os.path.join(output_dir, img)
        predicted_class_name = image_pose_light_classify(pose_model_path='yolo11x-pose.pt', det_model_path='runs/detect/train_light35deg/weights/best.pt', 
                                               img_path=img_path, output_path=output_path)
        
        if predicted_class_name == class_name:
            correct_classified_count += 1

        processed_count += 1

        if processed_count % 30 == 0:
            print(f'Processed {processed_count} of {total_images}')
    
    print(f'Correctly classified images: {correct_classified_count}')
    print(f'Class accuracy: {correct_classified_count / total_images}')

Processing class: left_outlet from test/test/left_outlet
Total images: 133
Processed 30 of 133
Processed 60 of 133
Processed 90 of 133
Processed 120 of 133
Correctly classified images: 126
Class accuracy: 0.9473684210526315
Processing class: right_outlet from test/test/right_outlet
Total images: 134
Processed 30 of 134
Processed 60 of 134
Processed 90 of 134
Processed 120 of 134
Correctly classified images: 134
Class accuracy: 1.0


In [16]:
pose_detect_batch_classify(input_dir='test/test', output_dir='pose_detect_images/test_posex', classes = ['left_AP', 'right_AP'], 
                           pose_model_path='yolo11x-pose.pt', det_model_path='runs/detect/train_light35deg/weights/best.pt')

Processing class: left_AP from test/test
Total images: 45
Processed 30 of 45
Correctly classified images: 45
Class accuracy: 1.0
Processing class: right_AP from test/test
Total images: 30
Processed 30 of 30
Correctly classified images: 30
Class accuracy: 1.0
Total processed: 75
Total correct: 75
Total Accuracy: 1.0


### Pose Estimation using Mediapipe ###

In [ ]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tatks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE
)

with PoseLandmarker.create_from_options(options) as landmarker:

In [ ]:
# convert input image to mediapipe.Image object
def create_mp_image('str')
mp_image = mp.Image.create_from_file('')